In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
from datetime import datetime, timedelta
np.random.seed(1)

In [0]:
from bs4 import BeautifulSoup
import requests
import time
import random
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [0]:
ks_2017_dataset = pd.read_csv('./data_2017/ks_2017.csv')

In [0]:
ks_2017_dataset['state'].value_counts()

successful    14595
failed        12271
Name: state, dtype: int64

In [0]:
url_ks_2017 = ks_2017_dataset['url_project']
url_ks_2017

0        https://www.kickstarter.com/projects/208113746...
1        https://www.kickstarter.com/projects/731341200...
2        https://www.kickstarter.com/projects/200651371...
3        https://www.kickstarter.com/projects/vgn/can-y...
4        https://www.kickstarter.com/projects/199408029...
                               ...                        
26861    https://www.kickstarter.com/projects/141645759...
26862    https://www.kickstarter.com/projects/claudiuss...
26863    https://www.kickstarter.com/projects/128754148...
26864    https://www.kickstarter.com/projects/716960512...
26865    https://www.kickstarter.com/projects/136026435...
Name: url_project, Length: 26866, dtype: object

In [0]:
options = ChromeOptions()
options.add_argument('headless')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36")
#path = '/Users/DaeyoungKim/Python/Final_Project/Kickstarter_2019-12-12/chromedriver' #chromedriver 위치
#driver = Chrome(executable_path=path, options=options) #컴퓨터에 Path 환경변수 설정 안 했을 때엔 여기서 이렇게 해줄 수 있음
driver = Chrome(options=options)

filename_contents = 'ks_content_crawled_2016.csv'
filename_errors = 'ks_middle_errors_2016.csv'
content_list = []

for idx, url in enumerate(url_ks_2016[23815:], start=23815):
    driver.get(url)
    time.sleep(random.uniform(8,20))
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    try:
        content_tag = soup.select_one('div.rte__content')
        contents = content_tag.select('p')
        contents_collected = []
        for c in contents:
            content = c.get_text().strip()
            contents_collected.append(content)
        
        try:
            risk_challenge_tag = soup.select_one('div#risksAndChallenges')
            risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
            for rc in risk_challenge_list:
                risk_challenge = rc.get_text().strip()
        except:
            risk_challenge=" " #또는 np.nan
        
        #오류 나기 전까지 저장되어 있는 content_list에는 이어서 append하고; 그래서 content_list에 모으는 것도 유지할 수 있도록
        content_list.append([idx, contents_collected, risk_challenge])
        
        #오류 난 후부턴 다시 크롤링 할 때, 매 페이지 긁어온 것마다 바로바로 csv파일에 한줄씩 append 할 것이다
        content_list_since_error = []
        content_list_since_error.append([idx, contents_collected, risk_challenge])
        pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
        print(idx)
    
    except Exception as ex:
        print("Unexpected error at {}".format(idx), ex)
        error_url = url
        count = 0
        while count < 4:
            print("Retrying:", count)
            driver.get(error_url)
            time.sleep(random.uniform(8,20))
            req = driver.page_source
            soup = BeautifulSoup(req, 'lxml')
            try:
                content_tag = soup.select_one('div.rte__content')
                contents = content_tag.select('p')
                contents_collected = []
                for c in contents:
                    content = c.get_text().strip()
                    contents_collected.append(content)
                
                try:
                    risk_challenge_tag = soup.select_one('div#risksAndChallenges')
                    risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
                    for rc in risk_challenge_list:
                        risk_challenge = rc.get_text().strip()
                except:
                    risk_challenge=" " #또는 np.nan
                
                content_list.append([idx, contents_collected, risk_challenge])
                
                content_list_since_error = []
                content_list_since_error.append([idx, contents_collected, risk_challenge])
                pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
                print(idx)
                break
            
            except Exception as et:
                print("Error while retrying:", et)
                time.sleep(random.uniform(100,130))
                count += 1
                continue
        
        #만약 재시도를 10번 했는데도 크롤링에 실패한다면, 원래 했던대로 오류 index, url, nan값을 csv파일들에 저장
        if count == 4:
            content_list.append([idx, error_url, np.nan])
            
            error_into_content_csv = []
            error_into_content_csv.append([idx, np.nan, np.nan])
            pd.DataFrame(error_into_content_csv, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
            
            error_list = []
            error_list.append([idx, error_url])
            pd.DataFrame(error_list, columns=['index','error_url']).to_csv(filename_errors, index=False, header=False, mode='a', encoding='UTF-8')
        
        continue

driver.close()

23815
23816
23817
23818
23819
23820
23821
23822
23823
23824
23825
23826
23827
23828
23829
23830
23831
23832
23833
23834
23835
23836
23837
23838
23839
23840
23841
23842
23843
23844
23845
23846
23847
23848
23849
23850
23851
23852
23853
23854
23855
23856
23857
23858
23859
23860
23861
23862
23863
23864
23865
23866
23867
23868
23869
23870
23871
23872
23873
23874
23875
23876
23877
23878
23879
23880
23881
23882
23883
23884
23885
23886
23887
23888
23889
23890
23891
23892
23893
23894
Unexpected error at 23895 'NoneType' object has no attribute 'select'
Retrying: 0
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 1
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 2
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 3
23895
23896
23897
23898
23899
23900
23901
23902
23903
23904
23905
23906
23907
23908
23909
23910
23911
23912
23913
23914
23915
23916
23917
23918
23919
23920
23921
23922
23923
23924
23925
23926
23927
23928


In [0]:
#2017년 추합

In [0]:
c1 = pd.read_csv('./data_2017/ks_content_crawled_2017_0-11000.csv')

In [0]:
c2 = pd.read_csv('./data_2017/ks_content_crawled_2017_11000_12000.csv')

In [0]:
c3 = pd.read_csv('./data_2017/ks_content_crawled_2017_12000_13000.csv')

In [0]:
c4 = pd.read_csv('./data_2017/ks_content_crawled_2017_13000_14000.csv')

In [0]:
c5 = pd.read_csv('./data_2017/ks_content_crawled_2017_14000_15000.csv')

In [0]:
c6 = pd.read_csv('./data_2017/ks_content_crawled_2017_15000-21000.csv')

In [0]:
c7 = pd.read_csv('./data_2017/ks_content_crawled_2017_21000_26866.csv')

In [0]:
concat_crawled_2017 = pd.concat([c1,c2,c3,c4,c5,c6,c7], ignore_index=True)

In [0]:
concat_crawled_2017.sort_values(by='index', inplace=True)

In [0]:
concat_crawled_2017.isna().sum()

index              0
content            4
risk_challenge    28
dtype: int64

In [0]:
errors_df = concat_crawled_2017[concat_crawled_2017['content'].str.startswith('http') == True]
errors_df

,index,content,risk_challenge
13291,13291,https://www.kickstarter.com/projects/793428750...,NaN
13385,13385,https://www.kickstarter.com/projects/manorroma...,NaN
13488,13488,https://www.kickstarter.com/projects/906553629...,NaN
13597,13597,https://www.kickstarter.com/projects/bingcheng...,NaN
13710,13710,https://www.kickstarter.com/projects/603457801...,NaN
13821,13821,https://www.kickstarter.com/projects/191855905...,NaN
13973,13973,https://www.kickstarter.com/projects/385567835...,NaN


In [0]:
erros_index = errors_df['index'].values
erros_index

array([13291, 13385, 13488, 13597, 13710, 13821, 13973], dtype=int64)

In [0]:
errors_df.to_csv('./data_2017/ks_errors_2017.csv', index=False, encoding='utf-8')

In [0]:
content_2017_errors_dropped = concat_crawled_2017.drop(erros_index)
content_2017_errors_dropped.reset_index(drop=True, inplace=True)

In [0]:
content_2017_errors_dropped

,index,content,risk_challenge
0,0,"['WHAT’S THE STORY?', ""Like you, we understand...",There’s always a degree of uncertainty with th...
1,1,"['About Us', ""Tyler (Director) is a recent gra...",Every film is going to have it's challenges. E...
2,2,['Boating is an enjoyable hobby. Being on the ...,Risks: \r\nI don't see any risks\r\n\r\nChalle...
3,3,"['An alien game-show which abducts six, seemin...",As this is not my first KickStarter comic book...
4,4,"['THE STORY', 'August Peyton was once a happil...","We need to raise £10,000 to complete principal..."
...,...,...,...
26854,26861,['Student finance league has 16000 students in...,The risks and challenges which come with our p...
26855,26862,['Please click cc for English subtitles in abo...,There is always an ever so slim chance that pr...
26856,26863,['This initiative was born of a desire to help...,This project carries with it little inherent r...
26857,26864,"['Hi,', 'My name is Katherine Gammon.', 'I am ...",The risks are very low. This is my final year ...


In [0]:
len(concat_crawled_2017)-len(erros_index)

26859

In [0]:
content_2017_errors_dropped.to_csv('./data_2017/ks_content_2017_recrawled_0_26865.csv', index=False, encoding='utf-8')

In [0]:
errors_df.reset_index(drop=True, inplace=True)

In [0]:
errors_df['content']

0    https://www.kickstarter.com/projects/793428750...
1    https://www.kickstarter.com/projects/manorroma...
2    https://www.kickstarter.com/projects/906553629...
3    https://www.kickstarter.com/projects/bingcheng...
4    https://www.kickstarter.com/projects/603457801...
5    https://www.kickstarter.com/projects/191855905...
6    https://www.kickstarter.com/projects/385567835...
Name: content, dtype: object

In [0]:
options = ChromeOptions()
options.add_argument('headless')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36")
#path = '/Users/DaeyoungKim/Python/Final_Project/Kickstarter_2019-12-12/chromedriver' #chromedriver 위치
#driver = Chrome(executable_path=path, options=options) #컴퓨터에 Path 환경변수 설정 안 했을 때엔 여기서 이렇게 해줄 수 있음
driver = Chrome(options=options)

filename_contents = './data_2017/ks_content_2017_recrawled_0_26865.csv'
filename_errors = './data_2017/new_errors_2017_recrawled.csv'

for row, url in enumerate(errors_df['content']):
    idx = errors_df['index'][row]
    
    driver.get(url)
    time.sleep(random.uniform(8,20))
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    try:
        content_tag = soup.select_one('div.rte__content')
        contents = content_tag.select('p')
        contents_collected = []
        for c in contents:
            content = c.get_text().strip()
            contents_collected.append(content)
        
        try:
            risk_challenge_tag = soup.select_one('div#risksAndChallenges')
            risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
            for rc in risk_challenge_list:
                risk_challenge = rc.get_text().strip()
        except:
            risk_challenge=" " #또는 np.nan

        content_list_since_error = []
        content_list_since_error.append([idx, contents_collected, risk_challenge])
        pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
        print("row:{}".format(row), "index:{}".format(idx))
        
    except Exception as ex:
        print("Unexpected error at {}".format(idx), ex)
        error_url = url
        count = 0
        while count < 2:
            print("Retrying:", count)
            driver.get(error_url)
            time.sleep(random.uniform(8,20))
            req = driver.page_source
            soup = BeautifulSoup(req, 'lxml')
            try:
                content_tag = soup.select_one('div.rte__content')
                contents = content_tag.select('p')
                contents_collected = []
                for c in contents:
                    content = c.get_text().strip()
                    contents_collected.append(content)
                
                try:
                    risk_challenge_tag = soup.select_one('div#risksAndChallenges')
                    risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
                    for rc in risk_challenge_list:
                        risk_challenge = rc.get_text().strip()
                except:
                    risk_challenge=" " #또는 np.nan
                
                content_list_since_error = []
                content_list_since_error.append([idx, contents_collected, risk_challenge])
                pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
                print("Retry successful")
                print("row:{}".format(row), "index:{}".format(idx))
                break
                
            except Exception as et:
                print("Error while retrying:", et)
                time.sleep(random.uniform(100,130))
                count += 1
                continue
        
        if count == 2:
            error_into_saved_csv = []
            error_into_saved_csv.append([idx, np.nan, np.nan])
            pd.DataFrame(error_into_saved_csv, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
            
            error_list = []
            error_list.append([idx, error_url])
            pd.DataFrame(error_list, columns=['index','error_url']).to_csv(filename_errors, index=False, header=False, mode='a', encoding='UTF-8')
            print("Retry failed")
            
        continue
    
driver.close()

row:0 index:13291
row:1 index:13385
row:2 index:13488
row:3 index:13597
row:4 index:13710
row:5 index:13821
row:6 index:13973


In [0]:
c_2017_recrawled = pd.read_csv('./data_2017/ks_content_2017_recrawled_0_26865.csv')

In [0]:
c_2017_recrawled.sort_values(by='index', inplace=True)

In [0]:
c_2017_recrawled.reset_index(drop=True,inplace=True)

In [0]:
c_2017_recrawled

,index,content,risk_challenge
0,0,"['WHAT’S THE STORY?', ""Like you, we understand...",There’s always a degree of uncertainty with th...
1,1,"['About Us', ""Tyler (Director) is a recent gra...",Every film is going to have it's challenges. E...
2,2,['Boating is an enjoyable hobby. Being on the ...,Risks: \r\nI don't see any risks\r\n\r\nChalle...
3,3,"['An alien game-show which abducts six, seemin...",As this is not my first KickStarter comic book...
4,4,"['THE STORY', 'August Peyton was once a happil...","We need to raise £10,000 to complete principal..."
...,...,...,...
26861,26861,['Student finance league has 16000 students in...,The risks and challenges which come with our p...
26862,26862,['Please click cc for English subtitles in abo...,There is always an ever so slim chance that pr...
26863,26863,['This initiative was born of a desire to help...,This project carries with it little inherent r...
26864,26864,"['Hi,', 'My name is Katherine Gammon.', 'I am ...",The risks are very low. This is my final year ...


In [0]:
c_2017_recrawled.to_csv('./data_2017/ks_content_2017_recrawled_0_26865.csv', index=False, encoding='utf-8')

In [0]:
c_2017_recrawled.isna().sum()

index              0
content            4
risk_challenge    21
dtype: int64